### Projetinho de Web Scraping para pegar letras de música


In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

#Alguns sites bloqueiam o acesso de bots, então é necessário simular um navegador
#Para isso, é necessário passar um cabeçalho com o user-agent do navegador
#O user-agent é uma string que identifica o navegador e o sistema operacional

def get_soup(url):
    headers = {
        'User-Agent': 'Mozilla/5.0', # Simulate a browser
        'Referer': 'https://www.google.com' # Diz qual a página de onde veio
    }
    req = Request(url, headers=headers)
    html = urlopen(req).read()
    return BeautifulSoup(html, 'html.parser')

In [10]:
#1. Coletar títulos e links de músicas de um artista

artist_url = 'https://www.letras.mus.br/ana-carolina/'
soup = get_soup(artist_url) # Pega o HTML da página e acessamos o artista

#Coletar os nomes das músicas mais tocadas e o link para a página da música
mais_tocadas_ana = []
html_aux = soup.find('ol', class_='cnt-list cnt-list--num artist-songList-content js-song-list')
for item in html_aux.find_all('li'):
    #print(item.text.strip())
    link = 'https://www.letras.mus.br' + item.a['href']
    titulo = item.text.strip()
    tupla = (titulo, link)
    mais_tocadas_ana.append(tupla)




In [11]:
for t in mais_tocadas_ana:
    print(t)

('Quem De Nós Dois (La Mia Storia Tra Le Dita)', 'https://www.letras.mus.br/ana-carolina/1397341/')
('Garganta', 'https://www.letras.mus.br/ana-carolina/44123/')
('Confesso', 'https://www.letras.mus.br/ana-carolina/confessa/')
('Rosas', 'https://www.letras.mus.br/ana-carolina/822662/')
('Encostar Na Tua', 'https://www.letras.mus.br/ana-carolina/75723/')
('Quem de Nós Dois', 'https://www.letras.mus.br/ana-carolina/44130/')
('Uma Louca Tempestade', 'https://www.letras.mus.br/ana-carolina/76008/')
('Problemas', 'https://www.letras.mus.br/ana-carolina/1960490/')
('Nua', 'https://www.letras.mus.br/ana-carolina/75827/')
('Eu Sei Que Vou Te Amar', 'https://www.letras.mus.br/ana-carolina/eu-sei-que-vou-te-amar/')
('Nada Pra Mim', 'https://www.letras.mus.br/ana-carolina/44124/')
('Ruas de Outono', 'https://www.letras.mus.br/ana-carolina/861120/')
('Vai', 'https://www.letras.mus.br/ana-carolina/861123/')
('Só de Sacanagem', 'https://www.letras.mus.br/ana-carolina/832783/')
('Pra Rua Me Levar', '

In [ ]:
#Coletar a letra de uma música - vamos visitar a página da música e extrair a letra completa
def get_lyrics(url):
    soup = get_soup(url) # Pega o HTML da página da música e transforma em um objeto
    letra_div = soup.find('div', class_='lyric-original')
    #print(letra_div.text)

    paragrafos = []
    for p in letra_div.find_all('p'):
        for br in p.find_all('br'): #está procurando a tag <br> dentro do parágrafo
            br.replace_with('\n')
        texto = p.text #pega o texto do parágrafo
        paragrafos.append(texto) #adiciona o texto na lista de parágrafos
    return '\n\n'.join(paragrafos) #retorna os parágrafos juntos em uma string, #separados por duas quebras de linha pois é o que o site usa para separar os parágrafos

titulo, link = mais_tocadas_ana[0]
letra = get_lyrics(link)
print(letra)

Eu e você
Não é assim tão complicado
Não é difícil perceber
Quem de nós dois
Vai dizer que é impossível
O amor acontecer

Se eu disser que já nem sinto nada
Que a estrada sem você é mais segura
Eu sei você vai rir da minha cara
Eu já conheço o teu sorriso, leio o teu olhar
Teu sorriso é só disfarce
Que eu já nem preciso

Sinto dizer
Que amo mesmo, tá ruim pra disfarçar
Entre nós dois
Não cabe mais nenhum segredo
Além do que já combinamos

No vão das coisas que a gente disse
Não cabe mais sermos somente amigos
E quando eu falo que eu já nem quero
A frase fica pelo avesso
Meio na contra-mão
E quando finjo que esqueço
Eu não esqueci nada

E cada vez que eu fujo, eu me aproximo mais
E te perder de vista assim é ruim demais
E é por isso que atravesso o teu futuro
E faço das lembranças um lugar seguro

Não é que eu queira reviver nenhum passado
Nem revirar um sentimento revirado
Mas toda vez que eu procuro uma saída
Acabo entrando sem querer na sua vida

Eu procurei qualquer desculpa pra não

In [24]:
#4. Montar um DataFrame com várias letras. Coletar título + link + letra de várias músicas e salvar em um DataFrame
import pandas as pd

data = []
for titulo, link in mais_tocadas_ana:
    letra = get_lyrics(link)
    row = {
        'titulo': titulo,
        'link': link,
        'letra': letra
        }
    data.append(row)

df = pd.DataFrame(data)
df.head()

,titulo,link,letra
0,Quem De Nós Dois (La Mia Storia Tra Le Dita),https://www.letras.mus.br/ana-carolina/1397341/,Eu e você\nNão é assim tão complicado\nNão é d...
1,Garganta,https://www.letras.mus.br/ana-carolina/44123/,Minha garganta estranha\nQuando não te vejo\nM...
2,Confesso,https://www.letras.mus.br/ana-carolina/confessa/,"Confesso, acordei achando tudo indiferente\nVe..."
3,Rosas,https://www.letras.mus.br/ana-carolina/822662/,Você pode me ver do jeito que quiser\nEu não v...
4,Encostar Na Tua,https://www.letras.mus.br/ana-carolina/75723/,Essa música foi removida em razão de solicitaç...


In [29]:
#6. Capturar as categorias do menu principal do site. Pegar o nome das categorias que aparecem no topo (ex: Artistas, Estilos, etc)
# é um menu lateral, que está dentro de nav

nav = soup.find("nav", class_="asideMenu-nav js-asideMenu-nav")

for a in nav.find_all("a"):
    print(a.text.strip())


Estilos musicais
Músicas
Artistas
Álbuns
Playlists
Blog
Treino de pronúncia
Aulas rápidas
Cursos de idiomas
Assinar o Letras
Enviar ou corrigir letras e mais


In [ ]:
#7. pegar todas as imagens da home. Listar os links das imagens que aparecem na home do site

home_url = 'https://www.letras.mus.br/'
soup = get_soup(home_url)

imgs = soup.find_all('img')

for img in imgs:
    print(img['src'])


https://akamai.sscdn.co/uploadfile/letras/imagem_destaques/10748_pt_BR_e154c90.jpg
https://akamai.sscdn.co/uploadfile/letras/fotos/3/0/8/3/3083b1621a661eee9c2eb4ef9d03daa5-tb5.jpg
https://akamai.sscdn.co/uploadfile/letras/fotos/e/d/8/1/ed81d6c11a327c91d479ee057e15a913-tb5.jpg
https://akamai.sscdn.co/uploadfile/letras/fotos/2/9/8/6/29866cf1c6051a0f685e358606c5e54a-tb5.jpg
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=
data:image/gif;base64,R0lGODlhAQABAA

In [34]:
#8. coletar e salvar localmente as imagens. Acessar a pag principal, coletar todos os img, filtrar os que tem url valida, baixar e salvar com nome organizado e criar uma pasta para salvar as imagens

import os
import time
from urllib.request import urlretrieve

os.makedirs('imagens', exist_ok=True)

imgs = soup.find_all('img')

for k, img in enumerate(imgs[:10]):
    src = img['src']
    if not src and not src.startswith('http'): #se a url não começa com http, não é uma url válida
        continue

    ext = os.path.splitext(src)[1]
    filename = f'imagens/img_{k}{ext}'

    try:
        urlretrieve(src, filename)
        print(f'{src} baixado com sucesso')
        time.sleep(1) #espera 1 segundo para não sobrecarregar o servidor
    except:
        print(f'Erro ao baixar {src}')





https://akamai.sscdn.co/uploadfile/letras/imagem_destaques/10748_pt_BR_e154c90.jpg baixado com sucesso
https://akamai.sscdn.co/uploadfile/letras/fotos/3/0/8/3/3083b1621a661eee9c2eb4ef9d03daa5-tb5.jpg baixado com sucesso
https://akamai.sscdn.co/uploadfile/letras/fotos/e/d/8/1/ed81d6c11a327c91d479ee057e15a913-tb5.jpg baixado com sucesso
https://akamai.sscdn.co/uploadfile/letras/fotos/2/9/8/6/29866cf1c6051a0f685e358606c5e54a-tb5.jpg baixado com sucesso
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs= baixado com sucesso
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs= baixado com sucesso
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs= baixado com sucesso
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs= baixado com sucesso
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs= baixado com sucesso
data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs= baixado com sucesso


In [38]:
#10. verificar total de músicas por artista
artistas = ['ana-carolina', 'marisa-monte', 'caetano-veloso', 'gilberto-gil', 'gal-costa', 'mamonas-assassinas']

def contar_musicas_artista(url):
    soup = get_soup(url)
    container = soup.find('div', class_='cnt-list--alp')
    musicas = container.find_all('li')
    return len(musicas)

for artista in artistas:
    url = f'https://www.letras.mus.br/{artista}/'
    count = contar_musicas_artista(url)
    print(f'{artista}: {count} músicas')


ana-carolina: 248 músicas
marisa-monte: 235 músicas
caetano-veloso: 720 músicas
gilberto-gil: 647 músicas
gal-costa: 566 músicas
mamonas-assassinas: 43 músicas


In [40]:
#11. análise detalhada de artistas musicais

dados = []
artista = ['ana-carolina', 'marisa-monte', 'caetano-veloso', 'gilberto-gil', 'gal-costa', 'mamonas-assassinas']

for nome in artista:
    link = f'https://www.letras.mus.br/{nome}/'
    soup = get_soup(link)
    total = contar_musicas_artista(link)
    numero_1 = soup.find('ol', class_='cnt-list cnt-list--num artist-songList-content js-song-list')
    dados.append({'Artista': nome, 'Total de músicas': total, 'Música mais tocada': numero_1.text.strip()})

df = pd.DataFrame(dados)
print(df)

              Artista  Total de músicas  \
0        ana-carolina               248   
1        marisa-monte               235   
2      caetano-veloso               720   
3        gilberto-gil               647   
4           gal-costa               566   
5  mamonas-assassinas                43   

                                  Música mais tocada  
0  Quem De Nós Dois (La Mia Storia Tra Le Dita)  ...  
1  Depois    Amor I Love You / Citação: Trecho Da...  
2  Você Não Me Ensinou a Te Esquecer    Sozinho  ...  
3  Palco    Aquele Abraço    Não Chore Mais    Sí...  
4  Lágrimas Negras    Palavras No Corpo    Eterna...  
5  Pelados Em Santos    1406    Uma Arlinda Mulhe...  
